# download, prepopulate all the csv into minio

In [1]:
import pandas as pd
# import pyarrow as pa
# import pyarrow.parquet as pq

In [2]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
from dask.distributed import Client, LocalCluster
import dask
from dask.distributed import get_task_stream

In [3]:
import time

In [4]:
print('pandas version: %s' % pd.__version__)
print('numpy version: %s' % np.__version__)
print('dask version: %s' % dask.__version__)

pandas version: 1.4.1
numpy version: 1.22.2
dask version: 2022.02.0


https://docs.databricks.com/_static/notebooks/koalas-benchmark-distributed-execution.html?_ga=2.216403934.95291449.1648935555-599276868.1645477063

In [17]:
filename = "taxi_dataset_2009-13.txt"

In [26]:
with open(filename) as file:
    csv_files = [line.rstrip() for line in file]
# only choose yellow taxis
yellow = list(filter(lambda x: "yellow" in x, csv_files))

In [27]:
# make the list small for now
# yellow = ['https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-11.csv']

In [28]:
# yellow

['https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-01.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-02.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-03.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-04.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-05.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-06.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-07.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-08.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-09.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-10.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-11.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-12.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2012-01.csv',
 'https://s3.amazonaws.co

In [21]:
from fsspec.registry import known_implementations
known_implementations

{'file': {'class': 'fsspec.implementations.local.LocalFileSystem'},
 'memory': {'class': 'fsspec.implementations.memory.MemoryFileSystem'},
 'dropbox': {'class': 'dropboxdrivefs.DropboxDriveFileSystem',
  'err': 'DropboxFileSystem requires "dropboxdrivefs","requests" and "dropbox" to be installed'},
 'http': {'class': 'fsspec.implementations.http.HTTPFileSystem',
  'err': 'HTTPFileSystem requires "requests" and "aiohttp" to be installed'},
 'https': {'class': 'fsspec.implementations.http.HTTPFileSystem',
  'err': 'HTTPFileSystem requires "requests" and "aiohttp" to be installed'},
 'zip': {'class': 'fsspec.implementations.zip.ZipFileSystem'},
 'tar': {'class': 'fsspec.implementations.tar.TarFileSystem'},
 'gcs': {'class': 'gcsfs.GCSFileSystem',
  'err': 'Please install gcsfs to access Google Storage'},
 'gs': {'class': 'gcsfs.GCSFileSystem',
  'err': 'Please install gcsfs to access Google Storage'},
 'gdrive': {'class': 'gdrivefs.GoogleDriveFileSystem',
  'err': 'Please install gdrivef

# Configure minio

In [22]:
import getpass

In [23]:


#tag::minio_storage_options[]
# straight from holden's example
minio_storage_options = {
    "key": getpass.getpass(),
    "secret": getpass.getpass(),
    "client_kwargs": {
        # "endpoint_url": "http://minio-1602984784.minio.svc.cluster.local:9000",
        # "endpoint_url": "http://minio.pigscanfly.ca:9000",
        "endpoint_url": "http://10.43.207.163:9000",
        "region_name": 'us-east-1'
    },
    "config_kwargs": {"s3": {"signature_version": 's3v4'}},
}
#end::minio_storage_options[]

 ····
 ···············································


In [8]:
!pip install pyarrow

  Using cached pyarrow-7.0.0-cp38-cp38-manylinux_2_17_aarch64.manylinux2014_aarch64.whl (24.9 MB)


In [12]:
# !conda install -c conda-forge s3fs

In [25]:
# !pip install fsspec[s3] aiohttp

In [9]:
import pyarrow

In [29]:
# yellow.pop()
# yellow.pop()
# yellow.pop()
yellow

['https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-01.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-02.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-03.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-04.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-05.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-06.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-07.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-08.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-09.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-10.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-11.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-12.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2012-01.csv',
 'https://s3.amazonaws.co

In [30]:
# yellow_backup = yellow

In [15]:
len(yellow)

57

In [31]:
import re
pattern = '/[^\/]*\.csv$/gm'

while len(yellow) > 0:
    try:
        csv_url = yellow.pop()
        print(csv_url)
    #     get the filename only so I can sort by month / year
        csv_name = re.findall(r"[^\/]*\.csv$",csv_url)[0]
        df = dd.read_csv(csv_url,dtype={'tolls_amount': 'float64',
                                       'tip_amount': 'float64',
                                        'Tip_Amt': 'float64',
                                       'Tolls_Amt': 'float64',
                                        'surcharge': 'float64',
                                        'store_and_fwd_flag': 'object'
                                       })

        df = df.repartition(npartitions=4)
        df.to_parquet(f's3://mika/dask_sandbox/nyc_{csv_name}',
                      compression="gzip",
                      storage_options=minio_storage_options, 
                      engine="pyarrow")
    except Exception as e:
        print(f"failed, {csv_url}")
        print(e)

https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-09.csv
failed, https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-09.csv
Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+-----------+---------+----------+
| Column    | Found   | Expected |
+-----------+---------+----------+
| surcharge | float64 | int64    |
+-----------+---------+----------+

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={'surcharge': 'float64'}

to the call to `read_csv`/`read_table`.

Alternatively, provide `assume_missing=True` to interpret
all unspecified integer columns as floats.
https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-08.csv
failed, https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-08.csv
Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+-----------+---------+----------+
| Column    | Found   | Expected |
+-----------+---------+----------+
| surcharge

/opt/conda/lib/python3.8/site-packages/dask/dataframe/io/csv.py:181: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


failed, https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2010-12.csv
Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+--------------------+--------+----------+
| Column             | Found  | Expected |
+--------------------+--------+----------+
| store_and_fwd_flag | object | float64  |
+--------------------+--------+----------+

The following columns also raised exceptions on conversion:

- store_and_fwd_flag
  ValueError("could not convert string to float: 'N'")

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={'store_and_fwd_flag': 'object'}

to the call to `read_csv`/`read_table`.
https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2010-11.csv


/opt/conda/lib/python3.8/site-packages/dask/dataframe/io/csv.py:181: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/opt/conda/lib/python3.8/site-packages/dask/dataframe/io/csv.py:181: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/opt/conda/lib/python3.8/site-packages/dask/dataframe/io/csv.py:181: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/opt/conda/lib/python3.8/site-packages/dask/dataframe/io/csv.py:181: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/opt/conda/lib/python3.8/site-packages/dask/dataframe/io/csv.py:181: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/opt/conda

https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2010-10.csv
https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2010-09.csv
failed, https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2010-09.csv
Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+--------------------+---------+----------+
| Column             | Found   | Expected |
+--------------------+---------+----------+
| store_and_fwd_flag | object  | float64  |
| surcharge          | float64 | int64    |
+--------------------+---------+----------+

The following columns also raised exceptions on conversion:

- store_and_fwd_flag
  ValueError("could not convert string to float: 'N'")

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={'store_and_fwd_flag': 'object',
       'surcharge': 'float64'}

to the call to `read_csv`/`read_table`.
https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2010-08.csv
failed, https://s3.

...

https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-03.csv


/opt/conda/lib/python3.8/site-packages/dask/dataframe/io/csv.py:181: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/opt/conda/lib/python3.8/site-packages/dask/dataframe/io/csv.py:181: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/opt/conda/lib/python3.8/site-packages/dask/dataframe/io/csv.py:181: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/opt/conda/lib/python3.8/site-packages/dask/dataframe/io/csv.py:181: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/opt/conda/lib/python3.8/site-packages/dask/dataframe/io/csv.py:181: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/opt/conda

https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-02.csv


/opt/conda/lib/python3.8/site-packages/dask/dataframe/io/csv.py:181: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/opt/conda/lib/python3.8/site-packages/dask/dataframe/io/csv.py:181: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/opt/conda/lib/python3.8/site-packages/dask/dataframe/io/csv.py:181: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/opt/conda/lib/python3.8/site-packages/dask/dataframe/io/csv.py:181: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-01.csv


https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-10.csv


list

s3://mika/dask_sandbox/nyc_yellow_tripdata_2009-10.csv


/opt/conda/lib/python3.8/site-packages/dask/dataframe/io/csv.py:181: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/opt/conda/lib/python3.8/site-packages/dask/dataframe/io/csv.py:181: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/opt/conda/lib/python3.8/site-packages/dask/dataframe/io/csv.py:181: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/opt/conda/lib/python3.8/site-packages/dask/dataframe/io/csv.py:181: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/opt/conda/lib/python3.8/site-packages/dask/dataframe/io/csv.py:181: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/opt/conda

(None,)